# HW4: Multi-Layer Neural Network
## Nicholas Thomson

### Import Libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Import Dataset

Dataset was obtained from: https://www.kaggle.com/datasets/jahnveenarang/cvdcvd-vd

The dataset contains information about whether a customer made a purchase after clicking an online ad.

In [2]:
# Load the dataset
data = pd.read_csv("Social_Network_Ads.csv")

# Display the first few rows of the dataset
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


### Preprocess the Data

I will remove User ID from the data, as this has no correlation to purchasing a product. The data will contain Gender, Age and Estimated Salary as the independent variables. Purchased will be the dependent variable.

In [3]:
# Split the data into features (X) and target variable (y)
X = data.drop(['Purchased', 'User ID'], axis=1)
#Males and females are dummy variables
X = pd.get_dummies(X, dtype=float)
y = data['Purchased']
X.head()

,Age,EstimatedSalary,Gender_Female,Gender_Male
0,19,19000,0.0,1.0
1,35,20000,0.0,1.0
2,26,43000,1.0,0.0
3,27,57000,1.0,0.0
4,19,76000,0.0,1.0


In [4]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype('float32'))
X_test_tensor = torch.tensor(X_test.astype('float32'))
y_train_tensor = torch.tensor(y_train.values.astype('float32')).unsqueeze(1) 
# Using y_train.values is necessary because PyTorch tensors expect NumPy arrays as input, not pandas Series objects.
# X_train has already been converted to Numpy array by the scaler
y_test_tensor = torch.tensor(y_test.values.astype('float32')).unsqueeze(1)

### Define the neural network architecture

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)  # Input layer to hidden layer
        self.fc2 = nn.Linear(16, 8)           # Hidden layer to hidden layer
        self.fc3 = nn.Linear(8, 1)            # Hidden layer to output layer
        self.relu = nn.ReLU()                  # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Pass through first hidden layer
        x = self.relu(self.fc2(x))  # Pass through second hidden layer
        x = torch.sigmoid(self.fc3(x))  # Pass through output layer with sigmoid activation
        return x

### Initialize the Neural Network

In [6]:
# Instantiate the neural network, creating a new instance of the neural network model.
input_size = X_train_tensor.shape[1]  # Number of features
model = NeuralNetwork(input_size)

### Define Loss Function and Optimizer

In [7]:
# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

### Train the Model

In [8]:
# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print progress
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the training set
with torch.no_grad():
    outputs = model(X_train_tensor)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_train_tensor).sum().item() / len(y_train_tensor)
    print(f'Accuracy on training set: {accuracy:.4f}')

Epoch [100/1000], Loss: 0.5235
Epoch [200/1000], Loss: 0.3308
Epoch [300/1000], Loss: 0.2631
Epoch [400/1000], Loss: 0.2403
Epoch [500/1000], Loss: 0.2308
Epoch [600/1000], Loss: 0.2243
Epoch [700/1000], Loss: 0.2205
Epoch [800/1000], Loss: 0.2177
Epoch [900/1000], Loss: 0.2148
Epoch [1000/1000], Loss: 0.2127
Accuracy on training set: 0.9187


### Evaluate the Model

In [9]:
# Evaluate the model on the test set
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy on test set: {accuracy:.4f}')

Accuracy on test set: 0.9250


The model is fairly accurate, able to classify 92% of cases correctly. This model could be useful in algorithms deciding on whether to display a certain ad or not. This could help target ads based on user information.